In [265]:
import pandas as pd
from sklearn.datasets import make_regression
from sklearn.multioutput import MultiOutputRegressor,MultiOutputClassifier
from sklearn.ensemble import GradientBoostingRegressor
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import make_scorer
from sklearn.preprocessing import StandardScaler


df = pd.read_csv('train.csv',header=None)
y_train1 = df.iloc[:,294:]
x_train1= df.iloc[:,:294]

X_train, X_test, y_train, y_test = train_test_split( x_train1, y_train1, test_size=0.2, random_state=42)

#y_test1= MultiOutputRegressor(GradientBoostingRegressor(min_samples_split=25,learning_rate=0.15,max_depth=4, min_samples_leaf=10,random_state=10,n_estimators=22)).fit(X_train, y_train).predict(X_test)

#y_test1= MultiOutputRegressor(SVR(C=60,kernel='linear')).fit(X_train, y_train).predict(X_test)



In [266]:
pipe_svr = Pipeline([('scl', StandardScaler()),
        ('reg', MultiOutputRegressor(SVR(C=350)))])

pipe_svc = Pipeline([('scl', StandardScaler()),
        ('clas', MultiOutputClassifier(SVC(C=250)))])

In [267]:
y_test1=pipe_svr.fit(X_train, y_train).predict(X_test)

In [268]:
y_test1=np.around(y_test1, decimals=0)
y_test1=y_test1.astype(np.int64)


In [269]:
accuracy_score(y_test,y_test1)

0.59375

In [271]:
from sklearn.model_selection import cross_validate

scoring = {'ac': make_scorer(accuracy_score)}

cross_validate(pipe_svr, x_train1, y_train1, scoring=scoring,cv=4, return_train_score=True)

ValueError: Classification metrics can't handle a mix of multilabel-indicator and continuous-multioutput targets

In [ ]:
df_test= pd.DataFrame(data=y_test)
#np.savetxt("foo.csv", y_test, delimiter=",")

df_test.to_csv('prediction.csv',index=False,header=False)